# Feature Extraction

Since the filters have been explored, the data displayed in spectrograms can now be written to `csv`-files for later training of the NN. The amount of data per soundfile is calculated with the following formula:

```
size_in_b = n_frames * n_bands * sizeof(float32)
```

This is one of the major strenghts of this method: instead of storing `512` bins `1024` times per second, the octave bank only creates `31` bins `24` times a second. 

In [52]:
import os
from octafilt3r import filter as o3f
import librosa as lr
import numpy as np
import json


user_mode = False

In [53]:
if user_mode == 'y':
    txt_name = input("give your storage-file a name: ") + '.txt'
else:
    txt_name = "th_oct_feature.txt"

std_txt_fp = "Datasets/Feature_extraction/th_oct_coeffs.txt"
source_dir = 'Datasets/UrbanSoundMini/'

fs = 48000                              # sample rate
fmin = 100                              # lowest f of interest
fmax = 12800                            # highest f of interest
oct_bw_ratio = 1/3                      # octave / amount of subdivisions
order = 8                               # order of octave filters

n_coeffs_p_section = 6                  # constant, definition of sos-matrix
n_sections = int(order/2)               # number of biquads (order 2 filters)
dec_stages = 4                          # number of decimations across bank
dec_ord = 10                            # order of decimation AA-filters

frame_size = 1000                      # framing size
fps = int(fs / frame_size)
window_size = fs * 1
max_dur = 5                            # max duration of sound in s
_, _, _, n_bands = o3f._gen_fc_fl_fu(fmax, fmin, oct_bw_ratio)

param_dict_fp = 'param_dict.json'
param_dict = {  
    "txt name":txt_name,
    "std_txt_fp":std_txt_fp,
    "source_dir":source_dir,
    "fs":fs,
    "fmin":fmin,
    "fmax":fmax,
    "oct_bw_ratio":oct_bw_ratio,
    "order":order,
    "n_coeffs_p_section":n_coeffs_p_section,
    "n_sections":n_sections,
    "dec_stages":dec_stages,
    "dec_ord":dec_ord,
    "frame_size": frame_size,
    "fps":fps,
    "window_size": window_size,
    "max_dur":max_dur,
    "n_bands":n_bands
}

with open(param_dict_fp, 'w') as f:
    json.dump(param_dict, f)

In [55]:
files_ignored = 0
tot_cnt = 0

df = []
labels = []
for class_ in os.listdir(source_dir):
    cnt = 1
    for filename in os.listdir(f'{source_dir}{class_}'):

        print(f'Currently extracting features in "{class_}" from {filename} (# {cnt})')

        wav = f'{source_dir}{class_}/{filename}'
        y, sr = lr.load(wav, sr=fs, mono=True, duration=max_dur)

        wins = int(len(y)/window_size)
        if wins == 0:
            files_ignored += 1
            print('*ignored, file under 1 second long*')
            continue    # skip files which are shorter than 1s

        # obtain features of whole file
        feats, fcs = o3f.rolling_oct_bank(y, fs, ratio=oct_bw_ratio, fmax=fmax, fmin=fmin, frame_size=frame_size, n_decimations=dec_stages)

        print(f'\tSplitting "{filename}" into {wins} 1-second part(s)')
        for win in range(wins):    # ignore parts of file which is not in a full window
            cur_feats = feats[fps * win:fps * (win + 1)]
            df.append(cur_feats)
            labels.append(class_)
        cnt += 1

    tot_cnt += cnt

df = np.dstack(df)
df = np.transpose(df, (2, 0, 1))

df_2d = df.reshape(df.shape[0], -1)
np.savetxt("Datasets/Feature_extraction/th_oct_feature.txt", df_2d)
with open("Datasets/Feature_extraction/labels.txt", "w") as f:
    for i in labels:
        f.writelines(i + '\n')


print(f'----- done. -----\nTotal file count: {tot_cnt}\nFiles ignored: {files_ignored}\nExtracted {len(df)} windows from {tot_cnt-files_ignored} Files')
print(f'Shape of dataframe: {df.shape}')

Currently extracting features in "car_horn" from 199769-1-0-1.wav (# 1)
	Splitting "199769-1-0-1.wav" into 4 1-second part(s)
Currently extracting features in "car_horn" from 199769-1-0-10.wav (# 2)
	Splitting "199769-1-0-10.wav" into 4 1-second part(s)
Currently extracting features in "car_horn" from 199769-1-0-11.wav (# 3)
	Splitting "199769-1-0-11.wav" into 4 1-second part(s)
Currently extracting features in "car_horn" from 199769-1-0-12.wav (# 4)
	Splitting "199769-1-0-12.wav" into 4 1-second part(s)
Currently extracting features in "car_horn" from 199769-1-0-13.wav (# 5)
	Splitting "199769-1-0-13.wav" into 4 1-second part(s)
Currently extracting features in "car_horn" from 199769-1-0-14.wav (# 6)
	Splitting "199769-1-0-14.wav" into 4 1-second part(s)
Currently extracting features in "car_horn" from 199769-1-0-15.wav (# 7)
	Splitting "199769-1-0-15.wav" into 4 1-second part(s)
Currently extracting features in "car_horn" from 199769-1-0-16.wav (# 8)
	Splitting "199769-1-0-16.wav" in